In [85]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, make_pipeline
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import seaborn as sns
import math
from scipy.stats import ks_2samp
from scipy.stats import ttest_ind

import warnings
warnings.filterwarnings("ignore")
matplotlib.rcParams.update({'font.size': 16})

#from jupyterthemes import jtplot
#jtplot.style(theme='monokai', context='notebook', ticks=True, grid = False)

random_state = 42

In [86]:
data = pd.read_csv('./pricing4.csv')

In [87]:
data['overpriced'] = (data['y_true - y_pred']/data['price']).apply(lambda x: 1 if x>0.1 else -1)

In [88]:
corr = data.corr()
corr.overpriced.sort_values(ascending = False).head(40)

overpriced                   1.000000
y_true - y_pred              0.346944
price                        0.208923
property_type_1              0.067349
public_transport             0.065258
mega_minor_dest              0.048083
mega_night_life              0.043565
minor_dest                   0.043038
food                         0.038171
property_type_21             0.037107
night_life                   0.035719
mega_public_transport        0.034655
mega_food                    0.033385
beds                         0.030457
review_scores_cleanliness    0.029188
major_dest                   0.026514
mega_major_dest              0.026514
room_type_1                  0.025530
accommodates                 0.024159
property_type_7              0.021640
metropolitan_3               0.021431
Unnamed: 0                   0.021267
property_type_17             0.020951
metropolitan_4               0.018574
cancellation_policy_2        0.013315
bedrooms                     0.012702
bed_type_5  

ok now smth else

In [89]:
data2 = np.load('./ratios.npy')

In [90]:
df2 = pd.DataFrame

In [91]:
ids = pd.read_csv('./ids.csv')

In [92]:
ids

,Unnamed: 0,id
0,0,2746729
1,1,12286328
2,2,156926
3,3,3767793
4,4,3645277
...,...,...
22393,22393,15860751
22394,22394,1312336
22395,22395,17302014
22396,22396,18477878


In [93]:
data['id'] = ids['id']

In [94]:
data

,Unnamed: 0,accommodates,bathrooms,bedrooms,beds,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,...,property_type_32,property_type_33,property_type_34,room_type_0,room_type_1,room_type_2,price,y_true - y_pred,overpriced,id
0,0,4,1.0,1.0,2.0,10.0,10.0,10.0,9.0,96.0,...,0,0,0,1,0,0,125,23.116386,1,2746729
1,1,6,2.5,1.0,6.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,24,-134.827000,-1,12286328
2,2,6,2.5,1.0,6.0,10.0,9.0,9.0,9.0,93.0,...,0,0,0,0,0,1,24,-44.417328,-1,156926
3,3,10,2.5,4.0,5.0,9.0,8.0,9.0,8.0,78.0,...,0,0,0,1,0,0,105,-259.051820,-1,3767793
4,4,4,1.0,1.0,2.0,10.0,10.0,10.0,9.0,93.0,...,0,0,0,0,1,0,85,13.085320,1,3645277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22393,22393,10,2.0,3.0,4.0,10.0,7.0,9.0,10.0,70.0,...,0,0,0,1,0,0,129,-141.791900,-1,15860751
22394,22394,4,1.0,1.0,3.0,10.0,9.0,10.0,9.0,91.0,...,0,0,0,1,0,0,115,16.101143,1,1312336
22395,22395,2,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,80,-37.915833,-1,17302014
22396,22396,4,0.5,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,38,-118.019440,-1,18477878


In [95]:
d = {}
for i in range(len(data2[0])):
    d[data2[0][i]] = data2[1][i]

In [96]:
data['occupied'] = data['id'].apply(lambda z: d[z])

In [97]:
data

,Unnamed: 0,accommodates,bathrooms,bedrooms,beds,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,...,property_type_33,property_type_34,room_type_0,room_type_1,room_type_2,price,y_true - y_pred,overpriced,id,occupied
0,0,4,1.0,1.0,2.0,10.0,10.0,10.0,9.0,96.0,...,0,0,1,0,0,125,23.116386,1,2746729,0.624658
1,1,6,2.5,1.0,6.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,24,-134.827000,-1,12286328,0.989041
2,2,6,2.5,1.0,6.0,10.0,9.0,9.0,9.0,93.0,...,0,0,0,0,1,24,-44.417328,-1,156926,0.967123
3,3,10,2.5,4.0,5.0,9.0,8.0,9.0,8.0,78.0,...,0,0,1,0,0,105,-259.051820,-1,3767793,0.923288
4,4,4,1.0,1.0,2.0,10.0,10.0,10.0,9.0,93.0,...,0,0,0,1,0,85,13.085320,1,3645277,0.915068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22393,22393,10,2.0,3.0,4.0,10.0,7.0,9.0,10.0,70.0,...,0,0,1,0,0,129,-141.791900,-1,15860751,0.849315
22394,22394,4,1.0,1.0,3.0,10.0,9.0,10.0,9.0,91.0,...,0,0,1,0,0,115,16.101143,1,1312336,0.857534
22395,22395,2,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,80,-37.915833,-1,17302014,0.750685
22396,22396,4,0.5,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,38,-118.019440,-1,18477878,0.046575


In [98]:
data['overpriced'].corr(data['occupied'])

0.08664847857319515